In [1]:
from moviepy.editor import VideoFileClip, concatenate_videoclips, AudioFileClip, CompositeAudioClip
from moviepy.audio.AudioClip import concatenate_audioclips
import os
import random
from natsort import natsorted


In [2]:
local_language_name='nufi'
local_language_name='yoruba'

# local_language_name='lingala'
# local_language_name='swahili'

# local_language_name='igbo'

# local_language_name='kinyarwanda'
# local_language_name='swahili'
# local_language_name='lingala'
# local_language_name='fulfulde'
# local_language_name='chichewa'
# local_language_name='igbo'


lln = local_language_name.title()
lln 

'Yoruba'

In [3]:

# def process_chunk(chunk, ad_clip_path, instrumental_path, output_filename,
#                   fadein_instr=1, fadeout_instr=1, reduce_volume_by=0.5, 
#                   original_audio_volume=0.5, fadein_ad=2, fadeout_ad=2,
#                   remove_original_audio=False, include_instrumental=True):
#     if not chunk:
#         print("No video clips to process.")
#         return

#     # Load the advertisement clip
#     ad_clip = VideoFileClip(ad_clip_path)
#     ad_clip_audio = ad_clip.audio.audio_fadein(fadein_ad).audio_fadeout(fadeout_ad)
#     ad_clip = ad_clip.set_audio(ad_clip_audio)

#     # Check if there are enough clips to split
#     mid_position = len(chunk) // 2
#     if mid_position == 0:
#         print("Not enough video clips to split into two parts.")
#         return


#     first_part = concatenate_videoclips(chunk[:mid_position])
#     second_part = concatenate_videoclips(chunk[mid_position:])

#     final_audio_first = final_audio_second = None

#     if include_instrumental:
#         # Load and process the instrumental audio file if included
#         instrumental_audio = AudioFileClip(instrumental_path)
#         instrumental_first = loop_audio(instrumental_audio, first_part.duration)
#         instrumental_second = loop_audio(instrumental_audio, second_part.duration)

#         # Apply volume and fade effects to instrumental audio
#         instrumental_first = (instrumental_first.volumex(reduce_volume_by)
#                               .audio_fadein(fadein_instr).audio_fadeout(fadeout_instr))
#         instrumental_second = (instrumental_second.volumex(reduce_volume_by)
#                                .audio_fadein(fadein_instr).audio_fadeout(fadeout_instr))

#     # Determine the final audio clips based on settings
#     if not remove_original_audio:
#         original_first = first_part.audio.volumex(original_audio_volume)
#         original_second = second_part.audio.volumex(original_audio_volume)
#         audio_components_first = [original_first] + ([instrumental_first] if include_instrumental else [])
#         audio_components_second = [original_second] + ([instrumental_second] if include_instrumental else [])
#         final_audio_first = CompositeAudioClip(audio_components_first)
#         final_audio_second = CompositeAudioClip(audio_components_second)
#     else:
#         final_audio_first = CompositeAudioClip([instrumental_first] if include_instrumental else [])
#         final_audio_second = CompositeAudioClip([instrumental_second] if include_instrumental else [])

#     # Set the processed audio clips to the corresponding video parts
#     first_part = first_part.set_audio(final_audio_first)
#     second_part = second_part.set_audio(final_audio_second)

#     # Combine all parts into the final clip
#     final_clip = concatenate_videoclips([first_part, ad_clip, second_part])

#     # Save the resulting video file
#     final_clip.write_videofile(output_filename, codec="libx264", fps=24)
#     print(f"Output saved to {output_filename}")

#     # Close all clips to free up memory
#     ad_clip.close()
#     first_part.close()
#     second_part.close()
#     final_clip.close()

In [4]:
from moviepy.editor import VideoFileClip, AudioFileClip, concatenate_videoclips, CompositeAudioClip

def process_chunk(chunk, ad_clip_path, instrumental_path, output_filename,
                  fadein_instr=1, fadeout_instr=1, reduce_volume_by=0.5, 
                  original_audio_volume=0.5, fadein_ad=2, fadeout_ad=2,
                  remove_original_audio=False, include_instrumental=True):
    if not chunk:
        print("No video clips to process.")
        return

    # Check if there are enough clips to split
    mid_position = len(chunk) // 2
    if mid_position == 0:
        print("Not enough video clips to split into two parts.")
        return

    first_part = concatenate_videoclips(chunk[:mid_position])
    second_part = concatenate_videoclips(chunk[mid_position:])

    final_audio_first = final_audio_second = None

    if include_instrumental:
        # Load and process the instrumental audio file if included
        instrumental_audio = AudioFileClip(instrumental_path)
        instrumental_first = loop_audio(instrumental_audio, first_part.duration)
        instrumental_second = loop_audio(instrumental_audio, second_part.duration)

        # Apply volume and fade effects to instrumental audio
        instrumental_first = (instrumental_first.volumex(reduce_volume_by)
                              .audio_fadein(fadein_instr).audio_fadeout(fadeout_instr))
        instrumental_second = (instrumental_second.volumex(reduce_volume_by)
                               .audio_fadein(fadein_instr).audio_fadeout(fadeout_instr))

    # Determine the final audio clips based on settings
    if not remove_original_audio:
        original_first = first_part.audio.volumex(original_audio_volume)
        original_second = second_part.audio.volumex(original_audio_volume)
        audio_components_first = [original_first] + ([instrumental_first] if include_instrumental else [])
        audio_components_second = [original_second] + ([instrumental_second] if include_instrumental else [])
        final_audio_first = CompositeAudioClip(audio_components_first)
        final_audio_second = CompositeAudioClip(audio_components_second)
    else:
        final_audio_first = CompositeAudioClip([instrumental_first] if include_instrumental else [])
        final_audio_second = CompositeAudioClip([instrumental_second] if include_instrumental else [])

    # Set the processed audio clips to the corresponding video parts
    first_part = first_part.set_audio(final_audio_first)
    second_part = second_part.set_audio(final_audio_second)

    # If ad_clip_path is provided, include the ad clip
    if ad_clip_path:
        ad_clip = VideoFileClip(ad_clip_path)
        ad_clip_audio = ad_clip.audio.audio_fadein(fadein_ad).audio_fadeout(fadeout_ad)
        ad_clip = ad_clip.set_audio(ad_clip_audio)
        final_clip = concatenate_videoclips([first_part, ad_clip, second_part])
    else:
        final_clip = concatenate_videoclips([first_part, second_part])

    # Save the resulting video file
    final_clip.write_videofile(output_filename, codec="libx264", fps=24)
    print(f"Output saved to {output_filename}")

    # Close all clips to free up memory
    if ad_clip_path:
        ad_clip.close()
    first_part.close()
    second_part.close()
    final_clip.close()


In [5]:
# Function to loop audio to required duration
def loop_audio(audio, duration):
    repeats = int(duration / audio.duration) + 1
    return concatenate_audioclips([audio] * repeats).subclip(0, duration)


# Path to the advertisements directory
ads_dir = fr'G:\My Drive\Data_Science\Resulam\Phrasebook_Audio_Video_Processing_production\Ads_Video\ResulamVideosAds_Bon\{lln}VideoAds'
ads_dir_global = fr'G:\My Drive\Data_Science\Resulam\Phrasebook_Audio_Video_Processing_production\Ads_Video\ResulamVideosAds_Bon\GlobalVideoAds'


try:

    # List the contents of both directories and create full paths
    video_ads_list = [os.path.join(ads_dir, f) for f in os.listdir(ads_dir)] + [os.path.join(ads_dir_global, f) for f in os.listdir(ads_dir_global)]
    video_ads_list = [i for i in video_ads_list if ".mp4" in i]
except:
    video_ads_list = [os.path.join(ads_dir_global, f) for f in os.listdir(ads_dir_global)]
    video_ads_list = [i for i in video_ads_list if ".mp4" in i]
    
video_ads_list = list(set(video_ads_list))  # Remove duplicates if any

video_ads_list = []


# Path to the chapters directories
chapters_dir = fr'G:\My Drive\Data_Science\Resulam\Phrasebook_Audio_Video_Processing_production\Languages\{lln}Phrasebook\bilingual_sentences'

instrumental_dir = r'G:\My Drive\Data_Science\Resulam\Phrasebook_Audio_Video_Processing_production\Background_music'

start_from_chapter = 1 
# Process each chapter
for chapter in natsorted(os.listdir(chapters_dir)):

    if "Chap" in chapter and "chunk" not in chapter:

        chap_num = int(chapter.split("Chap")[1])
        if chap_num >= start_from_chapter:
            print("Starting from Chapter: ",chap_num)

            chapter_path = os.path.join(chapters_dir, chapter)
            if os.path.isdir(chapter_path):
                print(f"Processing {chapter}...")
                # Naturally Sort Videos, and Take care of Special/Odd Videos containing the prefix "no_"
                all_videos = natsorted([v for v in os.listdir(chapter_path) if v.endswith(('.mp4', '.avi', '.mov'))])  # List all video files

                odd_sentences = [s for s in all_videos if "no_" in s]
                all_videos_to_sort = [s.split("no_",1)[-1] for s in all_videos]
                all_odd_sentences_suffix = [s.split("no_",1)[-1] for s in odd_sentences]
                all_videos_sorted = natsorted(all_videos_to_sort)
                all_videos_sorted = ["no_" + s if s in all_odd_sentences_suffix else s for s in all_videos_sorted]
            
                chunk_size = 20

                # Get the total number of videos
                len_total_videos = len(all_videos_sorted)

                for i in range(0, len_total_videos, chunk_size):
                    print(i)
                    chunk = all_videos_sorted[i:i+chunk_size] if i + chunk_size < len_total_videos else all_videos_sorted[i:]
                    video_clips = [VideoFileClip(os.path.join(chapter_path, video)) for video in chunk]
                    
                    # Randomly select an ad and an instrumental track
#                     ad_video = random.choice(os.listdir(ads_dir))
#                     ad_clip_path = os.path.join(ads_dir, ad_video)

                    if video_ads_list:
                        ad_clip_path = random.choice(video_ads_list)
                    else:
                        print("No video ads available.")
                        ad_clip_path = None
#                     ad_clip_path = random.choice(video_ads_list)
                    
                    instrumental_track = random.choice(os.listdir(instrumental_dir))
                    instrumental_path = os.path.join(instrumental_dir, instrumental_track)
                    
                    # Output filename
                    output_filename = os.path.join(chapters_dir, f"{chapter}_chunk_{(i//chunk_size) + 1}.mp4")
                    
                    # Process with settings
                    process_chunk(video_clips, ad_clip_path, instrumental_path, output_filename,
                                fadein_instr=2, fadeout_instr=2, reduce_volume_by=0.1, 
                                remove_original_audio=False, 
                                include_instrumental=True)

                    # Close video clips
                    for clip in video_clips:
                        clip.close()

Starting from Chapter:  1
Processing Chap1...
0
No video ads available.
Moviepy - Building video G:\My Drive\Data_Science\Resulam\Phrasebook_Audio_Video_Processing_production\Languages\YorubaPhrasebook\bilingual_sentences\Chap1_chunk_1.mp4.
MoviePy - Writing audio in Chap1_chunk_1TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video G:\My Drive\Data_Science\Resulam\Phrasebook_Audio_Video_Processing_production\Languages\YorubaPhrasebook\bilingual_sentences\Chap1_chunk_1.mp4



Moviepy - Done !
Moviepy - video ready G:\My Drive\Data_Science\Resulam\Phrasebook_Audio_Video_Processing_production\Languages\YorubaPhrasebook\bilingual_sentences\Chap1_chunk_1.mp4
Output saved to G:\My Drive\Data_Science\Resulam\Phrasebook_Audio_Video_Processing_production\Languages\YorubaPhrasebook\bilingual_sentences\Chap1_chunk_1.mp4
20
No video ads available.
Moviepy - Building video G:\My Drive\Data_Science\Resulam\Phrasebook_Audio_Video_Processing_production\Languages\YorubaPhrasebook\bilingual_sentences\Chap1_chunk_2.mp4.
MoviePy - Writing audio in Chap1_chunk_2TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video G:\My Drive\Data_Science\Resulam\Phrasebook_Audio_Video_Processing_production\Languages\YorubaPhrasebook\bilingual_sentences\Chap1_chunk_2.mp4



Moviepy - Done !
Moviepy - video ready G:\My Drive\Data_Science\Resulam\Phrasebook_Audio_Video_Processing_production\Languages\YorubaPhrasebook\bilingual_sentences\Chap1_chunk_2.mp4
Output saved to G:\My Drive\Data_Science\Resulam\Phrasebook_Audio_Video_Processing_production\Languages\YorubaPhrasebook\bilingual_sentences\Chap1_chunk_2.mp4
40
No video ads available.
Moviepy - Building video G:\My Drive\Data_Science\Resulam\Phrasebook_Audio_Video_Processing_production\Languages\YorubaPhrasebook\bilingual_sentences\Chap1_chunk_3.mp4.
MoviePy - Writing audio in Chap1_chunk_3TEMP_MPY_wvf_snd.mp3


...

Moviepy - Done !
Moviepy - video ready G:\My Drive\Data_Science\Resulam\Phrasebook_Audio_Video_Processing_production\Languages\YorubaPhrasebook\bilingual_sentences\Chap32_chunk_1.mp4
Output saved to G:\My Drive\Data_Science\Resulam\Phrasebook_Audio_Video_Processing_production\Languages\YorubaPhrasebook\bilingual_sentences\Chap32_chunk_1.mp4
20
No video ads available.
Moviepy - Building video G:\My Drive\Data_Science\Resulam\Phrasebook_Audio_Video_Processing_production\Languages\YorubaPhrasebook\bilingual_sentences\Chap32_chunk_2.mp4.
MoviePy - Writing audio in Chap32_chunk_2TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video G:\My Drive\Data_Science\Resulam\Phrasebook_Audio_Video_Processing_production\Languages\YorubaPhrasebook\bilingual_sentences\Chap32_chunk_2.mp4



Moviepy - Done !
Moviepy - video ready G:\My Drive\Data_Science\Resulam\Phrasebook_Audio_Video_Processing_production\Languages\YorubaPhrasebook\bilingual_sentences\Chap32_chunk_2.mp4
Output saved to G:\My Drive\Data_Science\Resulam\Phrasebook_Audio_Video_Processing_production\Languages\YorubaPhrasebook\bilingual_sentences\Chap32_chunk_2.mp4
40
No video ads available.
Moviepy - Building video G:\My Drive\Data_Science\Resulam\Phrasebook_Audio_Video_Processing_production\Languages\YorubaPhrasebook\bilingual_sentences\Chap32_chunk_3.mp4.
MoviePy - Writing audio in Chap32_chunk_3TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video G:\My Drive\Data_Science\Resulam\Phrasebook_Audio_Video_Processing_production\Languages\YorubaPhrasebook\bilingual_sentences\Chap32_chunk_3.mp4



Moviepy - Done !
Moviepy - video ready G:\My Drive\Data_Science\Resulam\Phrasebook_Audio_Video_Processing_production\Languages\YorubaPhrasebook\bilingual_sentences\Chap32_chunk_3.mp4
Output saved to G:\My Drive\Data_Science\Resulam\Phrasebook_Audio_Video_Processing_production\Languages\YorubaPhrasebook\bilingual_sentences\Chap32_chunk_3.mp4


In [6]:
ad_clip_path

# Version 2

In [7]:
# VERSION 2

In [8]:
# # [if "Chap" in i and "chunk" not in i ]

# for i in natsorted(os.listdir(chapters_dir)):

#     if "Chap" in i and "chunk" not in i:
#         chap_num = int(i.split("Chap")[1])
#         if chap_num >10:
#                     print(chap_num)
